# Financial assistance and alignment with China by country and country group

In [4]:
import plotly.express as px
data = px.data.iris()


from bokeh.plotting import figure, show, output_notebook
output_notebook(hide_banner=True)

import warnings
import pandas as pd
import numpy as np
import geopandas as gpd
from bokeh.models import (
    GeoJSONDataSource,
    Slider,
    CustomJS,
    LinearColorMapper,
    Range1d,
    ColumnDataSource,
    LabelSet,
    Span
)
from bokeh.models import HoverTool
from bokeh.layouts import column, row
import json
from bokeh.models import LinearAxis
from bokeh.palettes import PuBu, Greens, Reds
from IPython.display import display
from IPython.display import HTML

%pip install ipywidgets
%pip install colorcet
import colorcet
import ipywidgets as widgets


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Load world geometry data
world = gpd.read_file("data/ne_110m_admin_0_countries.shp")

AFR = ["Algeria","Angola","Benin","Botswana,Burkina Faso","Burundi","Cameroon","Congo","Côte d’Ivoire","Democratic Republic of the Congo","Djibouti","Egypt","Eritrea","Ethiopia","Gabon","The Gambia","Ghana","Kenya","Libya","Madagascar","Malawi","Mali","Mauritania","Mauritius","Morocco","Namibia","Nigeria","Rwanda","Senegal","Sierra Leone","South Africa","Somalia","Sudan","Togo","Tunisia","Uganda","Zambia",]
APAC = ["Afghanistan","Azerbaijan","Bahrain","Bangladesh","China","Fiji","India","Indonesia","Iraq","Japan","Jordan","Kazakhstan","Kuwait","Kyrgyzstan","Malaysia","Maldives","Marshall Islands","Mongolia","Nepal","Pakistan","Philippines","Qatar","Republic of Korea","Saudi Arabia","Sri Lanka","Thailand","United Arab Emirates","Uzbekistan","Viet Nam",]
GRULAC = ["Argentina","Bahamas","Bolivia (Plurinational State of)","Brazil","Chile","Costa Rica","Cuba","Ecuador","El Salvador","Guatemala","Honduras","Mexico","Nicaragua","Panama","Paraguay","Peru","Uruguay","Venezuela (Bolivarian Republic of)",]
WEOG = ["Australia","Austria","Belgium","Canada","Denmark","Finland","France","Germany","Iceland","Ireland","Italy","Luxembourg","Netherlands","Norway","Portugal","Spain","Switzerland","United Kingdom of Great Britain and Northern Ireland","United States of America",]
EG = ["Albania","Armenia","Bosnia and Herzegovina","Bulgaria","Czechia","Croatia","Estonia","Georgia","Hungary","Latvia","Lithuania","Montenegro","Poland","Republic of Moldova","Republic of North Macedonia","Romania","Russian Federation","Slovakia","Slovenia","Ukraine",]

# Function to assign color based on country group
def assign_color(country):
    if country in AFR:
        return "blue"
    elif country in APAC:
        return "green"
    elif country in GRULAC:
        return "red"
    elif country in WEOG:
        return "yellow"
    elif country in EG:
        return "orange"
    else:
        return "lightgray"


def get_country_group(country):
    if country in AFR:
        return "AFR"
    elif country in APAC:
        return "APAC"
    elif country in GRULAC:
        return "GRULAC"
    elif country in WEOG:
        return "WEOG"
    elif country in EG:
        return "EG"


def calculate_alignment(row):
    if row["Country"] == "China":
        return 1
    if row["China Vote"] == "Abstaining":
        return 0
    if row["Vote"] == row["China Vote"]:
        return 1
    else:
        return -1


def add_alignment_percentage(df):
    try:
        df["alignment_percentage"] = (
            df["alignment_score"] / df["Number of votes"]
        ) * 100
    except Exception:
        pass
    return df


def add_alignment(df):
    try:
        df["alignment"] = df["alignment_score"] / df["Number of votes"]
    except Exception:
        pass
    return df

def add_alignment_change(df):
    try:
        df["alignment_change"] = df["alignment"] - df["alignment"].shift(1)
    except Exception:
        pass
    return df

def add_fa_change(df):
    try:
        df["FT_ODI_change"] = df["FT_ODI"] - df["FT_ODI"].shift(1)
        df["AEI_FDI_change"] = df["AEI_FDI"] - df["AEI_FDI"].shift(1)
        df["BU_ODA_change"] = df["BU_ODA"] - df["BU_ODA"].shift(1)
        df["ODI_MOFCOM_change"] = df["ODI_MOFCOM"] - df["ODI_MOFCOM"].shift(1)
    except Exception:
        pass
    return df

def add_average_fa(df):
    try:
        df["Average_FA"] = (
            df[["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"]]
            .replace(0, np.nan)
            .mean(axis=1, skipna=True)
        )
        df["Average_FA"] = df["Average_FA"].fillna(0)
    except Exception:
        pass
    return df

def add_average_fa_change(df):
    try:
        df["Average_FA_change"] = df["Average_FA"] - df["Average_FA"].shift(1)
    except Exception:
        pass
    return df

def add_measures(df):
    df = add_alignment_percentage(df)
    df = add_alignment(df)
    df = add_average_fa(df)
    return df

def filter_country_group(row, country_group):
    return get_country_group(row["Country"]) == country_group


# Load and clean data:

# Load your data
oda = pd.read_csv("data/oda.csv")
votes = pd.read_csv("data/vote.csv")
df = pd.concat([oda, votes], ignore_index=True)
df_vac = pd.read_csv("data/vac.csv")


# Clean vaccine data
df_vac.fillna(0, inplace=True)
df_vac.rename(
    columns={
        "Destination": "Country",
        "Entry Date (date of delivery)": "Year",
        "Arrived (D)": "Quantity",
    },
    inplace=True,
)
df_vac = df_vac.drop(columns=["Region"])
df_vac["Quantity"] = df_vac["Quantity"].replace("-", np.nan)
df_vac.dropna(axis=0, subset=["Year", "Quantity"], inplace=True)
df_vac = df_vac[df_vac["Quantity"] != 0]
df_vac["Year"] = df_vac["Year"].replace("Aug", "01-Aug-21")
df_vac["Year"] = df_vac["Year"].replace("Oct-21", "01-Oct-21")
df_vac["Year"] = df_vac["Year"].replace("17 May 21", "17-May-21")
df_vac.head()
df_vac["Year"] = pd.to_datetime(df_vac["Year"], format="%d-%b-%y", dayfirst=True).dt.year
df_vac = df_vac.astype({"Year": "int", "Quantity": "float32"})
df_vac = df_vac.groupby(["Year", "Country"]).sum().reset_index()

# Clean vote data
df["Country Group"] = df["Country"].apply(get_country_group)
china_votes = df[df["Country"] == "China"][
    ["Session number", "Text title", "Vote"]
].rename(columns={"Vote": "China Vote"})
df = df.merge(
    china_votes,
    on=["Session number", "Text title"],
    suffixes=("", "_china"),
    how="left",
)
# Create a new column 'mapped_vote' that maps three values to -1, 0, and 1
vote_mapping = {"Against": -1, "Abstaining": 0, "In Favour": 1}
df["mapped_vote"] = df["Vote"].map(vote_mapping)
df["mapped_china_vote"] = df["China Vote"].map(vote_mapping)

df["alignment_score"] = df.apply(calculate_alignment, axis=1)
df_votes = (
    df.groupby(["Year", "Country", "Country Group", "Topic"])
    .agg(
        {
            "alignment_score": "sum",
            "Text title": "count",
        }
    )
    .reset_index()
)
df_votes.rename({"Text title": "Number of votes"}, axis=1, inplace=True)

# Clean FA data
df_fa = (
    df.groupby(["Year", "Country"])
    .agg(
        {
            "ODI_MOFCOM": "first",
            "BU_ODA": "first",
            "AEI_FDI": "first",
            "FT_ODI": "first",
        }
    )
    .reset_index()
)
df_fa[["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"]] = df_fa[
    ["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"]
].fillna(0)


# Global widgets
def get_filter(name):
    match name:
        case "fa_type":
            return widgets.Dropdown(
                options=["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM", "Average_FA"],value="FT_ODI",
                description="FA Type:",
            )
        case "country_group":
            return widgets.Dropdown(
                options=["All", "AFR", "APAC", "GRULAC", "WEOG", "EG"], description="Country Group:", value="All"
            )
        case "country":
            return widgets.Dropdown(options=df.sort_values("Country")["Country"].unique(), description="Country:", value=df["Country"].unique().tolist()[0])
        case "topic":
            return widgets.Dropdown(
                options=["All"]+ df["Topic"].unique().tolist() ,
                description="Topic:",value="All"
            )
        
        
optionlist = {
    'fa_type': ["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM", "Average_FA"],
    "country_group": ["All", "AFR", "APAC", "GRULAC", "WEOG", "EG"],
    "country": df.sort_values("Country")["Country"].unique(),
    "topic": ["All"]+ df["Topic"].unique().tolist()
}

In [6]:
# Voting alignment with China and financial assistance, over time, per country

import math
fa = df_fa
votes = df_votes
def get_label_percentage(x):
    if math.isnan(x):
        return 0
    return str(round(x,2))+"%"

def get_label(x):
    if math.isnan(x):
        return 0
    return str(round(x))

def get_alignment_color(x):
    if x <0:
        return Reds[7][2]
    return Greens[7][2]

def plot(fa_type, country):
    plot = None
    group_by = {"Year", "Country"}
    filtered_votes = votes
    filtered_fa = fa
    years = pd.DataFrame({"Year": df["Year"].unique()})

    if country == "All":
        group_by.remove("Country")
    else:
        filtered_votes = filtered_votes[filtered_votes["Country"]== country]
        filtered_fa = filtered_fa[filtered_fa["Country"]==country]
    
    filtered_votes = filtered_votes.groupby(list(set(filtered_votes.columns) & group_by)).agg(
        {
            "alignment_score": "sum",
            "Number of votes": "sum",
        }
    ).reset_index()
    filtered_fa = filtered_fa.groupby(list(set(filtered_fa.columns) & group_by)).agg(
        {
            "FT_ODI": "sum",
            "AEI_FDI": "sum",
            "BU_ODA": "sum",
            "ODI_MOFCOM": "sum",
        }
    ).reset_index()
    filtered_fa = add_measures(filtered_fa)
    filtered_votes = add_measures(filtered_votes)
    filtered_fa = years.merge(filtered_fa, on=["Year"], how="left")
    filtered_votes = years.merge(filtered_votes, on=["Year"], how="left")
    x = years["Year"].to_numpy()
    source = ColumnDataSource(
        data=dict(
            x=x,
            fa=filtered_fa[fa_type],
            fa_labels=filtered_fa[fa_type].apply(get_label),
            alignment_percentage=filtered_votes["alignment_percentage"],
            alignment_color=filtered_votes["alignment_percentage"].apply(get_alignment_color),
            alignment_label = filtered_votes["alignment_percentage"].apply(get_label_percentage)
        )
    )
    labels = LabelSet(
        x="x", y="fa", text="fa_labels", x_offset=-15, y_offset=5, source=source
    )
    labels2 = LabelSet(
        x="x", y="alignment_percentage",y_range_name="percent", text="alignment_label", x_offset=-35, y_offset=10, source=source
    )
    plot = figure(width=950, height=600, title="FA vs Alignment per country")
    plot.add_layout(labels)
    plot.add_layout(labels2)
    plot.y_range = Range1d(
        start=-(filtered_fa[fa_type].max() + filtered_fa[fa_type].max() / 2), end=filtered_fa[fa_type].max() + filtered_fa[fa_type].max() / 2
    )
    plot.extra_y_ranges = {"percent": Range1d(start=-125, end=125)}
    plot.add_layout(
        LinearAxis(y_range_name="percent", axis_label="Alignment Percentage"),
        "right",
    )
    plot.vbar(
        x=x,
        top=filtered_fa[fa_type],
        width=0.9,
        fill_color=PuBu[7][2],
        legend_label=f"Financial assistance:{fa_type}",
    )
    plot.circle(
        x="x",
        y="alignment_percentage",
        size=10,
        fill_color="alignment_color",
        y_range_name="percent",
        legend_label="Alignment %",
        source=source
    )
    plot.xaxis.ticker = list(range(x.min(), x.max()))

    show(plot)

from IPython.display import display, Markdown, Latex

fa_titles = {
    "FT_ODI": "Outward Direct Investment (Financial Times)",
    "ODI_MOFCOM": "Outward Direct Investment (China Ministry of Commerce)",
    "AEI_FDI": "Foreign Direct Investment (American Enterprise Institute)",
    "BU_ODA": "Official Development Assistance (Boston University)",
}

# fa_type = get_filter("fa_type")
# country_group = get_filter("country_group")
# topic = get_filter("topic")
for b in optionlist['fa_type']:
    for c in optionlist['country']:
        display(Markdown('## %s vs %s' % (fa_titles.get(b, b),c)))
        plot(b,c)


## FT_ODI vs Afghanistan

## FT_ODI vs Albania

## FT_ODI vs Algeria

## FT_ODI vs Angola

## FT_ODI vs Argentina

...

## AEI_FDI vs Mauritius

## AEI_FDI vs Mexico

## AEI_FDI vs Mongolia

## AEI_FDI vs Montenegro

## AEI_FDI vs Morocco